In [1]:
import japanize_matplotlib

# plot
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import yfinance as yf
from backtesting import Backtest, Strategy
from backtesting.lib import crossover
from stockstats import StockDataFrame

sns.set(font="IPAexGothic", rc={"figure.figsize": (11, 8)})
pd.options.display.float_format = "{:6.2f}".format

/usr/local/lib/python3.9/site-packages/backtesting/_plotting.py:50: UserWarning: Jupyter Notebook detected. Setting Bokeh output to notebook. This may not work in Jupyter clients without JavaScript support (e.g. PyCharm, Spyder IDE). Reset with `backtesting.set_bokeh_output(notebook=False)`.
  warnings.warn('Jupyter Notebook detected. '


Loading BokehJS ...

In [2]:
# Valid start and end: YYYY-MM-DD
# Valid periods: 1d,5d,1mo,3mo,6mo,1y,2y,5y,10y,ytd,max
# Valid intervals: [1m, 2m, 5m, 15m, 30m, 60m, 90m, 1h, 1d, 5d, 1wk, 1mo, 3mo]
response = yf.download(
    tickers="SOXL",
    period="1y",
    interval="1d",
    group_by="ticker",
)

[*********************100%***********************]  1 of 1 completed


In [3]:
yfdata = response.copy().dropna()
yfdata = yfdata["1950-01":"202１-12"]  # 直近の暴落を除いて検証する
yfdata

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2021-05-17,30.98,31.10,29.43,31.10,31.09,19224800
2021-05-18,31.71,32.08,30.22,30.29,30.28,15262600
2021-05-19,28.46,32.17,28.22,32.05,32.04,29208700
2021-05-20,32.75,34.98,32.67,34.64,34.63,28593200
2021-05-21,35.30,35.34,33.81,34.14,34.13,25100300
...,...,...,...,...,...,...
2021-12-27,68.10,73.00,68.00,72.99,72.98,15598000
2021-12-28,74.07,74.07,69.57,70.52,70.51,14401800
2021-12-29,70.42,72.23,69.60,71.00,70.99,10702000


In [4]:
def convert_df_to_stock_df(df: pd.DataFrame) -> StockDataFrame:
    sdf = df.copy()
    sdf.rename(
        columns={
            "Open": "open",
            "High": "high",
            "Low": "low",
            "Close": "close",
            "Adj Close": "amount",
            "Volume": "volume",
        },
        inplace=True,
    )
    sdf.index.names = ["date"]
    return StockDataFrame(sdf)


def MACD(
    arr: pd.DataFrame, ema_short_period: int, ema_long_period: int, signal: int
) -> tuple[pd.Series, pd.Series]:  # Macd  # Macd
    sdf = convert_df_to_stock_df(arr)
    StockDataFrame.MACD_EMA_SHORT = 12
    StockDataFrame.MACD_EMA_LONG = 26
    StockDataFrame.MACD_EMA_SIGNAL = 9
    return (sdf["macd"], sdf["macds"])

In [5]:
class SmaCross(Strategy):
    short = 12
    long = 26
    signal = 9

    count = 0

    def init(self):
        self.macd, self.macd_signal = self.I(
            MACD, self.data.df, self.short, self.long, self.signal
        )

    def next(self):
        self.count += 1

        # MACDが計算できていない場合トレードしない
        if self.count < self.long:
            return

        if crossover(self.macd, self.macd_signal):
            self.buy()

        elif crossover(self.macd_signal, self.macd):
            self.position.close()


bt = Backtest(yfdata, SmaCross, cash=10000, commission=0.002, exclusive_orders=True)

In [6]:
# 出力
output = bt.run()
print(output)
bt.plot()

Start                     2021-05-17 00:00:00
End                       2021-12-31 00:00:00
Duration                    228 days 00:00:00
Exposure Time [%]                       44.38
Equity Final [$]                     13655.17
Equity Peak [$]                      15819.56
Return [%]                              36.55
Buy & Hold Return [%]                  118.68
Return (Ann.) [%]                       63.34
Volatility (Ann.) [%]                   64.90
Sharpe Ratio                             0.98
Sortino Ratio                            2.51
Calmar Ratio                             4.49
Max. Drawdown [%]                      -14.11
Avg. Drawdown [%]                      -10.03
Max. Drawdown Duration       86 days 00:00:00
Avg. Drawdown Duration       35 days 00:00:00
# Trades                                    5
Win Rate [%]                            40.00
Best Trade [%]                          55.13
Worst Trade [%]                         -7.44
Avg. Trade [%]                    

Row(id='1493', ...)

In [ ]:
# 最適化
optimize = bt.optimize(
    short=range(3, 50, 3),
    long=range(3, 50, 3),
    signal=range(3, 50, 3),
    method="grid",  # unuse model-based optimization
    constraint=lambda p: p.short < p.long,
    maximize="Equity Final [$]",
)
bt.plot()
print(optimize)
print(optimize._strategy)

/usr/local/lib/python3.9/site-packages/backtesting/backtesting.py:1488: UserWarning: Searching for best of 1920 configurations.
  output = _optimize_grid()
